In [63]:
import requests
import selenium
from selenium import webdriver
import pyquery
#import tesserocr  
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import bs4
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium import webdriver
import time
import wget
import random
import time
from selenium.webdriver.common.by import By


In [105]:
driver = webdriver.ChromiumEdge()


In [106]:
#driver.get(driver.current_url)
driver.get('https://www.douyin.com/video/7196942004889161015')

In [34]:

def GetPage():
    '''
    This function gets the page source of the current page, refresh, and get the video link.
    '''
    driver.get(driver.current_url)
    time.sleep(2)  # Ensure the page has loaded
    html = driver.page_source

    global SoupedHtml
    SoupedHtml = BeautifulSoup(html, features='html.parser')

    pattern_video = re.compile('<source class="" src="//www..*?type="', re.S)
    found_videos = re.findall(pattern_video, str(SoupedHtml))

    if found_videos:
        # Process the found video links
        df_pattern_video_with_redundancy = pd.DataFrame(found_videos)
        df_pattern_video = pd.DataFrame(df_pattern_video_with_redundancy[0].apply(lambda x: x[24:-8]).tolist())
        
        global DownloadLink
        DownloadLink = "http://" + str(df_pattern_video.loc[0, 0])
        print("Found video link: " + DownloadLink)
    else:
        print("No video links found.")
        DownloadLink = None

    return(DownloadLink)


In [42]:
# Next Page Button Click
def NextPage():
    '''
    Press the next page button of the current Douyin Page
    '''
    actions = ActionChains(driver)
    #actions.click().perform()
    ActionChains(driver).key_down(Keys.DOWN).send_keys("s").perform()
    return

In [44]:
#Download the video
def Download():
    '''
    Download the video from the current Douyin Page to the local disk
    '''
    CurrentTime = time.strftime("%Y%m%d_%H:%M:%S", time.localtime()).replace(' ','')
    DownloadPath='/Users/zhanglu/Desktop/CODE/Formal/Video/'+ str(CurrentTime) + '.mp4'
    wget.download(DownloadLink,DownloadPath)
    return

In [111]:
def GetUserIDbyXpath():
    '''
    Get the user ID by Xpath
    '''
    UserID = driver.find_element(By.XPATH, '//*[@id="douyin-right-container"]/div[2]/div/div[1]/div[4]/div/div[1]/div[2]/a/div/span/span/span/span/span/span')

    return(UserID.text)

In [96]:
def GetPostMetadatabyclass():
    '''
    Get the number of hearts of the post by class name
    '''
    metadata = driver.find_elements(By.CLASS_NAME, 'ofo4bP_8')
    heartAmount = metadata[0].text
    commentAmount = metadata[1].text
    starAmount = metadata[2].text
    return(heartAmount, commentAmount, starAmount)

In [99]:
def GetRepostAmountbyclass():
    '''
    Get the number of reposts of the post by class name
    '''
    repostAmount = driver.find_element(By.CLASS_NAME, 'njfMvuRG').text
    return(repostAmount)

In [115]:
def GetHashTag():
     res = re.search('<span>#.*?</span>',str(SoupedHtml))
     if res:
          PatternHashTag = re.compile('<span>#.*?</span>',re.S)
          DF_PatternHashTag_With_Redundancy = pd.DataFrame(re.findall(PatternHashTag, str(SoupedHtml)))
          DF_PatternHashTag = pd.DataFrame(DF_PatternHashTag_With_Redundancy[0].apply(lambda x:x[6:-7]).tolist())
          return(DF_PatternHashTag)
     else:
          DF_PatternHashTag = pd.DataFrame(columns=[0])
          return(DF_PatternHashTag)


In [117]:
def GetContent():
    if  re.search('<span> [\u4e00-\u9fa5].*?[\u4e00-\u9fa5]</span>',str(SoupedHtml)):
        PatternContent = re.compile('<span> [\u4e00-\u9fa5].*?[\u4e00-\u9fa5]</span>',re.S)
        DF_PatternContent_With_Redundancy = pd.DataFrame(re.findall(PatternContent, str(SoupedHtml)))
        DF_PatternContent = pd.DataFrame(DF_PatternContent_With_Redundancy[0].apply(lambda x:x[6:-7]).tolist())
        return(DF_PatternContent)
    else:
        if re.search('<span><span>[\u4e00-\u9fa5].*?[\u4e00-\u9fa5]</span>',str(SoupedHtml)):
            PatternContent = re.compile('<span><span>[\u4e00-\u9fa5].*?[\u4e00-\u9fa5]</span>',re.S) 
            DF_PatternContent_With_Redundancy = pd.DataFrame(re.findall(PatternContent, str(SoupedHtml)))
            DF_PatternContent = pd.DataFrame(DF_PatternContent_With_Redundancy[0].apply(lambda x:x[12:-7]).tolist())
            return(DF_PatternContent)
    
        else:
            DF_PatternContent = pd.DataFrame(columns=[0])
            return(DF_PatternContent)

In [100]:
InitialPage('https://www.douyin.com/user/MS4wLjABAAAAVdf_7XGwDc5ZfviTdfJJPC8M8-E2MzWud1x0QGBbIuWvYY_AoiJsnJSQ8x6eKIIz?modal_id=7108307771870743839')


()

In [111]:
InitialPage('https://www.douyin.com/user/MS4wLjABAAAAVdf_7XGwDc5ZfviTdfJJPC8M8-E2MzWud1x0QGBbIuWvYY_AoiJsnJSQ8x6eKIIz?modal_id=7073446314352200995&relation=0')

()

In [102]:
InitialPage('https://www.douyin.com/hot?modal_id=7123759333241883943')

()

In [103]:
'''for i in range(1,1000):
    time.sleep(int(random.randint(2,69)))
    GetPage()
    Download()
    time.sleep(int(random.randint(2,69)))
    NextPage()'''

'for i in range(1,1000):\n    time.sleep(int(random.randint(2,69)))\n    GetPage()\n    Download()\n    time.sleep(int(random.randint(2,69)))\n    NextPage()'

In [104]:
def OutputCSV():
    GetUserID()
    GetUserFans()
    GetUserLikes()
    GetCommentsNumber()
    GetCollects()
    GetContent()
    GetHashTag()
    data_csv=pd.read_csv(r'/Users/zhanglu/Desktop/CODE/Formal/Video/Info.csv')
    temp2=pd.DataFrame({CurrentTime:[GetUserID(),GetUserFans(),GetUserLikes(),GetCommentsNumber(),GetCollects(),GetContent()]})
    temp3 = pd.concat([temp2,GetHashTag().rename(columns={0:CurrentTime})], ignore_index = True)
    data_csv[str(CurrentTime)]=temp3
    data_csv.to_csv('/Users/zhanglu/Desktop/CODE/Formal/DouyinRankingData/Info.csv',index=False,header=str(CurrentTime))
    return

In [123]:
GetUserFans()

KeyError: 0

In [127]:
for i in range(1,1000):
    CurrentTime = time.strftime("%Y%m%d_%H"+"."+"%M"+"."+"%S", time.localtime()).replace(' ','')
    GetPage()
    GetUserID()
    GetUserFans()
    GetUserLikes()
    GetCommentsNumber()
    GetCollects()
    GetContent()
    GetHashTag()
    data_csv=pd.read_csv(r'/Users/zhanglu/Desktop/CODE/Formal/DouyinRankingData/Info.csv')
    temp2=pd.DataFrame({CurrentTime:[GetUserID().loc[0,0],GetUserFans().loc[0,0],GetUserLikes().loc[0,0],GetCommentsNumber().loc[0,0],GetCollects().loc[0,0],GetContent().loc[0,0]]})
    temp3 = pd.concat([temp2,GetHashTag().rename(columns={0:CurrentTime})], ignore_index = True)
    data_csv[str(CurrentTime)]=temp3
    data_csv.to_csv('/Users/zhanglu/Desktop/CODE/Formal/DouyinRankingData/Info.csv',index=False,header=str(CurrentTime))
    
    DownloadPath='/Users/zhanglu/Desktop/CODE/Formal/Video/'+ str(CurrentTime) + '.mp4'
    wget.download(DownloadLink,DownloadPath)   
    #time.sleep(int(random.randint(2,69)))
    NextPage()


KeyError: 0